# ** Forever 24 Kobe Bryant and 2 Gigi Bryant **

<hr> 

- DNN for speech digit recognition 

In [1]:
import numpy as np 
import kaldi_io
from utils import *

In [2]:
targets_list = ['Z', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'O']
targets_mapping = {}
for i, x in enumerate(targets_list):
    targets_mapping[x] = i

# Layer层

In [3]:
class Layer(object):
    
    def forward(self, input):
        """Forward function by input
        Args:
            input: input, B * N matrix, B for batch size
        Returns:
            output when applied this layer
        """
        raise "Not implement error"
        
    def backward(self, input, output, d_output):
        """Compute gradient of this layer's input by (input, output, d_output)
           as well as compute the gradient of the parameter of this layer
        Args:
            input: input of this layer
            output: output of this layer
            d_output: accumulated gradient from final output to this
                      layer's output
        Returns:
            accumulated gradient from final output to this layer's input.
        """
        raise "Not implement error"
        
    def set_learning_rate(self, lr):
        """Set learning rate of this layer"""
        self.learning_rate = lr
        
    def update(self):
        """Update this layers parameter if it has or do nothing"""

# ReLU层

In [4]:
# ReLU类直接继承了Layer类
# ReLU: f(x) = max(0, x) = x if x_i > 0 else 0
class ReLU(Layer):
    
    def forward(self, Z):
        
        """
        Z -- output of the linear layer
        
        Returns:
        A -- Post-activation parameter, of the same shape as Z
        """
        A = np.maximum(0, Z)
        assert(A.shape == Z.shape)
        
        return A.T
                
    def backward(self, input, output, dA):
        
        # When z <= 0, set dz to 0. When z > 0, set dz to 1.
        dZ = np.array(dA, copy=True)
        dZ[input <= 0] = 0
#         dZ = 1.0 * (dZ > 0)
        assert(dZ.shape == input.shape)
        
        return dZ.T

In [5]:
def relu_forward_test():
    relu = ReLU()
    utt2feat, utt2target = read_feats_and_targets('train/feats.scp',
                                                      'train/text')
    inputs, labels = build_input(targets_mapping, utt2feat, utt2target)
    batch_size = 100
    _input = inputs[:batch_size]
    relu_forward = relu.forward(_input)

    print("|> _input shape is {}".format(_input.shape))
    print("|> ", relu_forward)
    print("|> relu forward result shape is {}".format(relu_forward.shape))

# relu_forward_test()

# FullyConnect层

In [6]:
class FullyConnect(Layer):
    
    def __init__(self, in_dim, hidden_dim):
        """
        in_dim -- size of the input layer
        hidden_dim -- size of the hidden layer
        """
        self.w = np.random.randn(hidden_dim, in_dim) * np.sqrt(2.0 / in_dim)
        self.b = np.zeros((hidden_dim, 1))
        self.dw = np.zeros((hidden_dim, in_dim))
        self.db = np.zeros((hidden_dim, 1))
        
    def forward(self, input):
        # Z = W · A + b
        # self.w shape is (out_dim, in_dim)
        # input shape is (batch_size, in_dim)
        Z = np.dot(self.w, input.T) + self.b
        assert Z.shape == (self.w.shape[0], input.shape[0])
        return Z
        
    def backward(self, A_prev, output, dZ):
        """
        dZ(hidden_dim, in_dim) -- Gradient of the cost respect to the linear output (of current layer L)
        
        Returns:
        dA_prev(batch_size, in_dim) -- Gradient of the cost with respect to the activation (of the previous layer L - 1), same shape as A_prev
        """
        batch_size = A_prev.shape[0]
        
        self.dw = np.dot(dZ, A_prev) / batch_size
        self.db = np.sum(dZ, axis=1, keepdims=True) / batch_size
        dA_prev = np.dot(self.w.T, dZ)
        
        assert (dA_prev.shape == A_prev.T.shape)
        assert (self.dw.shape == self.w.shape)
        assert (self.db.shape == self.b.shape)
        
        # Normalize dw/db by batch size
        self.dw = self.dw / batch_size
        self.db = self.db / batch_size
        return dA_prev.T
    
    def update(self):
        self.w = self.w - self.learning_rate * self.dw
        self.b = self.b - self.learning_rate * self.db

In [7]:
def FullyConnect_forward_test():
    utt2feat, utt2target = read_feats_and_targets('train/feats.scp',
                                                          'train/text')
    inputs, labels = build_input(targets_mapping, utt2feat, utt2target)
    batch_size = 100
    _input = inputs[:batch_size]
    
    # 429 is feature dim 
    # 128 is hidden dim 
    fullyconnect = FullyConnect(429, 128)
    return fullyconnect.forward(_input)
    
# FullyConnect_forward_test()

def FullyConnect_backward_test():
    utt2feat, utt2target = read_feats_and_targets('train/feats.scp',
                                                              'train/text')
    inputs, labels = build_input(targets_mapping, utt2feat, utt2target)
    batch_size = 100
    hidden_dim = 128
    in_dim = 429
    _input = inputs[:batch_size]

    np.random.seed(242)
    fullyconnect = FullyConnect(in_dim, hidden_dim)
    dZ = np.random.randn(hidden_dim, batch_size)
    return fullyconnect.backward(_input, _, dZ)

# FullyConnect_backward_test()

# Softmax 层

In [8]:
class Softmax(Layer):
    
    def forward(self, input):
        _input = input.T
        row_max = _input.max(axis=1).reshape(_input.shape[0], 1)
        x = _input - row_max
        return np.exp(x) / np.sum(np.exp(x), axis=1).reshape(x.shape[0], 1)
    
    def backward(self, input, output, d_output):
        """Directly return the d_output as we show below, the grad is to
            the activation(input) of softmax
        """
        return d_output

# DNN 层

In [9]:
class DNN(object):
    
    def __init__(self, in_dim, out_dim, hidden_dim, layer_nums):
        self.layers = []
        self.layers.append(FullyConnect(in_dim, hidden_dim))
        self.layers.append(ReLU())
        for i in range(layer_nums):
            self.layers.append(FullyConnect(hidden_dim, hidden_dim))
            self.layers.append(ReLU())
        self.layers.append(FullyConnect(hidden_dim, out_dim))
        self.layers.append(Softmax())
        
    def set_learning_rate(self, lr):
        for layer in self.layers:
            layer.set_learning_rate(lr)
            
    def forward(self, input):
        self.forward_buf = []
        out = input
        self.forward_buf.append(out)
        for i in range(len(self.layers)):
            out = self.layers[i].forward(out)
            self.forward_buf.append(out)
            
        assert (len(self.forward_buf) == len(self.layers) + 1)
        return out
    
    def backward(self, grad):
        """
        Args:
            grad: the grad is to the activation before softmax
        """
        self.backward_buf = [None] * len(self.layers)
        self.backward_buf[len(self.layers) - 1] = grad
        for i in range(len(self.layers) - 2, -1, -1):
            grad = self.layers[i].backward(self.forward_buf[i],
                                          self.forward_buf[i + 1],
                                          self.backward_buf[i + 1].T)
            self.backward_buf[i] = grad
        
    def update(self):
        for layer in self.layers:
            layer.update() 

In [10]:
def one_hot(labels, total_label):
    output = np.zeros((labels.shape[0], total_label))
    for i in range(labels.shape[0]):
        output[i][labels[i]] = 1.0
    return output

In [11]:
def dnn_forward_test():
    utt2feat, utt2target = read_feats_and_targets('train/feats.scp',
                                                              'train/text')
    inputs, labels = build_input(targets_mapping, utt2feat, utt2target)
    batch_size = 32
    permute = np.random.permutation(inputs.shape[0])
    inputs = inputs[permute]
    labels = labels[permute]
    _input = inputs[:batch_size]
    _label = labels[:batch_size]
    dnn = DNN(429, 11, 128, 1)
    dnn.set_learning_rate(1e-2)
    
    for i in range(len(dnn.layers)):
        print(dnn.layers[i])
    
    out = dnn.forward(_input)

    # out = dnn.layers[0].forward(_input)
    # out = dnn.layers[1].forward(out)
    # out = dnn.layers[2].forward(out)
    # out = dnn.layers[3].forward(out)
    # out = dnn.layers[4].forward(out)
    return out

# dnn_forward_test()

def dnn_backward_test():
    utt2feat, utt2target = read_feats_and_targets('train/feats.scp',
                                                              'train/text')
    inputs, labels = build_input(targets_mapping, utt2feat, utt2target)
    batch_size = 32
    permute = np.random.permutation(inputs.shape[0])
    inputs = inputs[permute]
    labels = labels[permute]
    _input = inputs[:batch_size]
    _label = labels[:batch_size]
    dnn = DNN(429, 11, 128, 1)
    dnn.set_learning_rate(1e-2)
    
    out = dnn.forward(_input)
    one_hot_label = one_hot(_label, out.shape[1])
    grad = out - one_hot_label
    
    dnn.backward_buf = [None] * len(dnn.layers)
    dnn.backward_buf[len(dnn.layers) - 1] = grad
    
    for i in range(len(dnn.layers) - 2, -1, -1):
        grad = dnn.layers[i].backward(dnn.forward_buf[i],
                                      dnn.forward_buf[i + 1],
                                      dnn.backward_buf[i + 1].T)
        dnn.backward_buf[i] = grad
        
    return dnn.backward_buf
    

In [12]:
def train(dnn):
    utt2feat, utt2target = read_feats_and_targets('train/feats.scp',
                                                  'train/text')
    inputs, labels = build_input(targets_mapping, utt2feat, utt2target)
    num_samples = inputs.shape[0]
    # shuffle data
    permute = np.random.permutation(num_samples)
    inputs = inputs[permute]
    labels = labels[permute]
    num_epochs = 100
    batch_size = 100
    for i in range(num_epochs):
        cur = 0
        while cur < num_samples:
            end = min(cur + batch_size, num_samples)
            input = inputs[cur:end]
            label = labels[cur:end]
            # Step1: forward
            out = dnn.forward(input)
            one_hot_label = one_hot(label, out.shape[1])
            # Step2: compute cross entropy loss and backward
            loss = -np.sum(np.log(out + 1e-20) * one_hot_label) / out.shape[0]
            # The grad is to activation before softmax
            grad = out - one_hot_label
            dnn.backward(grad)
            # Step3: update parameters
            dnn.update()
            print('Epoch {} num_samples {} loss {}'.format(i, cur, loss))
            cur += batch_size
            
def test(dnn):
    utt2feat, utt2target = read_feats_and_targets('test/feats.scp',
                                                  'test/text')
    total = len(utt2feat)
    correct = 0
    for utt in utt2feat:
        t = utt2target[utt]
        ark = utt2feat[utt]
        mat = kaldi_io.read_mat(ark)
        mat = splice(mat, 5, 5)
        posterior = dnn.forward(mat)
        posterior = np.sum(posterior, axis=0) / float(mat.shape[0])
        predict = targets_list[np.argmax(posterior)]
        if t == predict: correct += 1
        print('label: {} predict: {}'.format(t, predict))
    print('Acc: {:.3f}'.format(float(correct) / total))

In [13]:
def main():
    # for the memory of forever 24 Kobe and 2 Gigi.
    np.random.seed(242)
    # We splice the raw feat with left 5 frames and right 5 frames
    # So the input here is 39 * (5 + 1 + 5) = 429
    dnn = DNN(429, 11, 128, 1)
    dnn.set_learning_rate(5e-1)
    train(dnn)
    test(dnn)

if __name__ == '__main__':
    main()

Epoch 0 num_samples 0 loss 17.16087425558523
Epoch 0 num_samples 100 loss 18.30438824488586
Epoch 0 num_samples 200 loss 11.109657785227927
Epoch 0 num_samples 300 loss 8.886464483634008
Epoch 0 num_samples 400 loss 9.80744541644064
Epoch 0 num_samples 500 loss 7.9626844393742156
Epoch 0 num_samples 600 loss 8.812555081493828
Epoch 0 num_samples 700 loss 8.758557403710599
Epoch 0 num_samples 800 loss 8.442973745109903
Epoch 0 num_samples 900 loss 7.855848609177367
Epoch 0 num_samples 1000 loss 8.075113199222155
Epoch 0 num_samples 1100 loss 6.8369329396964895
Epoch 0 num_samples 1200 loss 6.988409854597134
Epoch 0 num_samples 1300 loss 6.5198158809919615
Epoch 0 num_samples 1400 loss 6.749792350708947
Epoch 0 num_samples 1500 loss 5.678345019414771
Epoch 0 num_samples 1600 loss 5.147394780225227
Epoch 0 num_samples 1700 loss 5.615616947627879
Epoch 0 num_samples 1800 loss 5.077236615397718
Epoch 0 num_samples 1900 loss 5.53220385510183
Epoch 0 num_samples 2000 loss 5.0421108039176445
E

Epoch 1 num_samples 400 loss 1.5474038851017429
Epoch 1 num_samples 500 loss 1.5984403468492001
Epoch 1 num_samples 600 loss 1.4948973080618948
Epoch 1 num_samples 700 loss 1.2951585958611307
Epoch 1 num_samples 800 loss 1.5283275528545572
Epoch 1 num_samples 900 loss 1.3392994141810244
Epoch 1 num_samples 1000 loss 1.3140029144778236
Epoch 1 num_samples 1100 loss 1.4118410560307615
Epoch 1 num_samples 1200 loss 1.098628448222979
Epoch 1 num_samples 1300 loss 1.3172736886436853
Epoch 1 num_samples 1400 loss 1.385394529816408
Epoch 1 num_samples 1500 loss 1.154712179140676
Epoch 1 num_samples 1600 loss 1.2188025101106057
Epoch 1 num_samples 1700 loss 1.391699178796202
Epoch 1 num_samples 1800 loss 1.1435096619677578
Epoch 1 num_samples 1900 loss 1.2039154016348086
Epoch 1 num_samples 2000 loss 1.4095783303307232
Epoch 1 num_samples 2100 loss 1.3513548141077598
Epoch 1 num_samples 2200 loss 1.30204736165637
Epoch 1 num_samples 2300 loss 1.234618109912567
Epoch 1 num_samples 2400 loss 1.5

Epoch 2 num_samples 1400 loss 1.0309334349721633
Epoch 2 num_samples 1500 loss 0.8033139370272173
Epoch 2 num_samples 1600 loss 0.878467211290895
Epoch 2 num_samples 1700 loss 0.93007150526612
Epoch 2 num_samples 1800 loss 0.7561219126091817
Epoch 2 num_samples 1900 loss 0.7823987929488623
Epoch 2 num_samples 2000 loss 0.9426038929472633
Epoch 2 num_samples 2100 loss 0.8365832456349215
Epoch 2 num_samples 2200 loss 0.7485533387834138
Epoch 2 num_samples 2300 loss 0.7822097818971604
Epoch 2 num_samples 2400 loss 0.9065545579343496
Epoch 2 num_samples 2500 loss 1.0188016535584825
Epoch 2 num_samples 2600 loss 0.8260637133911115
Epoch 2 num_samples 2700 loss 0.7778281736584931
Epoch 2 num_samples 2800 loss 0.9629512460827631
Epoch 2 num_samples 2900 loss 0.9509262151852539
Epoch 2 num_samples 3000 loss 0.7876712348217124
Epoch 2 num_samples 3100 loss 0.8091735679602868
Epoch 2 num_samples 3200 loss 1.0456896813823597
Epoch 2 num_samples 3300 loss 0.8107575294867192
Epoch 2 num_samples 340

Epoch 3 num_samples 2700 loss 0.6108219129878967
Epoch 3 num_samples 2800 loss 0.7497145701611692
Epoch 3 num_samples 2900 loss 0.6973764823386154
Epoch 3 num_samples 3000 loss 0.6005740037823244
Epoch 3 num_samples 3100 loss 0.62875693365673
Epoch 3 num_samples 3200 loss 0.826791145813057
Epoch 3 num_samples 3300 loss 0.6094001387387076
Epoch 3 num_samples 3400 loss 0.6350471089702758
Epoch 3 num_samples 3500 loss 0.6634920775581873
Epoch 3 num_samples 3600 loss 0.5185889742827918
Epoch 3 num_samples 3700 loss 0.6929982524830284
Epoch 3 num_samples 3800 loss 0.6599144459689055
Epoch 3 num_samples 3900 loss 0.7691633044242792
Epoch 3 num_samples 4000 loss 0.8095788662337171
Epoch 3 num_samples 4100 loss 0.82164262549603
Epoch 3 num_samples 4200 loss 0.587100681999503
Epoch 3 num_samples 4300 loss 0.6600179893339404
Epoch 3 num_samples 4400 loss 0.6963099238706296
Epoch 3 num_samples 4500 loss 0.755379627863126
Epoch 3 num_samples 4600 loss 0.6675774885251655
Epoch 3 num_samples 4700 lo

Epoch 4 num_samples 3800 loss 0.5231910691104126
Epoch 4 num_samples 3900 loss 0.6316119936623155
Epoch 4 num_samples 4000 loss 0.6853293720201404
Epoch 4 num_samples 4100 loss 0.6950897826066417
Epoch 4 num_samples 4200 loss 0.49401469242243623
Epoch 4 num_samples 4300 loss 0.5466444519464885
Epoch 4 num_samples 4400 loss 0.5574235274419543
Epoch 4 num_samples 4500 loss 0.6376974896125123
Epoch 4 num_samples 4600 loss 0.5461141331898137
Epoch 4 num_samples 4700 loss 0.3677375159872361
Epoch 4 num_samples 4800 loss 0.4598742675539755
Epoch 4 num_samples 4900 loss 0.570167385633141
Epoch 4 num_samples 5000 loss 0.698381999996073
Epoch 4 num_samples 5100 loss 0.6953489570121912
Epoch 4 num_samples 5200 loss 0.5331250625480919
Epoch 4 num_samples 5300 loss 0.5104641303507478
Epoch 4 num_samples 5400 loss 0.7626901982320902
Epoch 4 num_samples 5500 loss 0.5188530348763997
Epoch 4 num_samples 5600 loss 0.49802599499093786
Epoch 4 num_samples 5700 loss 0.7569638269205069
Epoch 4 num_samples 

...

Epoch 96 num_samples 3700 loss 0.008299534675626094
Epoch 96 num_samples 3800 loss 0.004442226385726906
Epoch 96 num_samples 3900 loss 0.005833101535074722
Epoch 96 num_samples 4000 loss 0.0060801114840429845
Epoch 96 num_samples 4100 loss 0.007696184662027948
Epoch 96 num_samples 4200 loss 0.0060647554744496165
Epoch 96 num_samples 4300 loss 0.005126883600029156
Epoch 96 num_samples 4400 loss 0.005909168271398215
Epoch 96 num_samples 4500 loss 0.00712772566271462
Epoch 96 num_samples 4600 loss 0.006435373840638502
Epoch 96 num_samples 4700 loss 0.0036125264702816986
Epoch 96 num_samples 4800 loss 0.003962463734404789
Epoch 96 num_samples 4900 loss 0.005091288130708188
Epoch 96 num_samples 5000 loss 0.004482088248502365
Epoch 96 num_samples 5100 loss 0.007828212576191942
Epoch 96 num_samples 5200 loss 0.0038640635928704824
Epoch 96 num_samples 5300 loss 0.005179635348425772
Epoch 96 num_samples 5400 loss 0.00718856063404141
Epoch 96 num_samples 5500 loss 0.004370084111639928
Epoch 96 n

Epoch 97 num_samples 3900 loss 0.005746957977088021
Epoch 97 num_samples 4000 loss 0.005997637530588535
Epoch 97 num_samples 4100 loss 0.0075579573394350865
Epoch 97 num_samples 4200 loss 0.00596158890294501
Epoch 97 num_samples 4300 loss 0.00503534412656642
Epoch 97 num_samples 4400 loss 0.005794181838649834
Epoch 97 num_samples 4500 loss 0.006982548461102396
Epoch 97 num_samples 4600 loss 0.00633165393117742
Epoch 97 num_samples 4700 loss 0.003552830490966501
Epoch 97 num_samples 4800 loss 0.003903472994304119
Epoch 97 num_samples 4900 loss 0.0050071519240303545
Epoch 97 num_samples 5000 loss 0.004404091710848471
Epoch 97 num_samples 5100 loss 0.007699086498149188
Epoch 97 num_samples 5200 loss 0.003806968813114846
Epoch 97 num_samples 5300 loss 0.005099137763749159
Epoch 97 num_samples 5400 loss 0.007073934412907987
Epoch 97 num_samples 5500 loss 0.004297141747548273
Epoch 97 num_samples 5600 loss 0.012981857775804725
Epoch 97 num_samples 5700 loss 0.005380077763589061
Epoch 97 num_

Epoch 98 num_samples 4400 loss 0.005695253590816985
Epoch 98 num_samples 4500 loss 0.006857519299382631
Epoch 98 num_samples 4600 loss 0.006226229831244431
Epoch 98 num_samples 4700 loss 0.003489471843491867
Epoch 98 num_samples 4800 loss 0.0038300134677337464
Epoch 98 num_samples 4900 loss 0.0049351345844107055
Epoch 98 num_samples 5000 loss 0.004336001096416619
Epoch 98 num_samples 5100 loss 0.007570382350425567
Epoch 98 num_samples 5200 loss 0.0037485494720160663
Epoch 98 num_samples 5300 loss 0.005012141471257111
Epoch 98 num_samples 5400 loss 0.006962458691250872
Epoch 98 num_samples 5500 loss 0.004231812013736776
Epoch 98 num_samples 5600 loss 0.01276960636793946
Epoch 98 num_samples 5700 loss 0.005287511169769066
Epoch 98 num_samples 5800 loss 0.005285278709686738
Epoch 98 num_samples 5900 loss 0.006551596354823399
Epoch 98 num_samples 6000 loss 0.005567686201399793
Epoch 98 num_samples 6100 loss 0.00515387505654757
Epoch 98 num_samples 6200 loss 0.005636637815286782
Epoch 98 nu

Epoch 99 num_samples 4900 loss 0.0048506084877778
Epoch 99 num_samples 5000 loss 0.004271938194473777
Epoch 99 num_samples 5100 loss 0.007439306878806975
Epoch 99 num_samples 5200 loss 0.003678305078294011
Epoch 99 num_samples 5300 loss 0.00492938687030525
Epoch 99 num_samples 5400 loss 0.006844995901266808
Epoch 99 num_samples 5500 loss 0.004162113493881437
Epoch 99 num_samples 5600 loss 0.01249116732970653
Epoch 99 num_samples 5700 loss 0.0051935562957588855
Epoch 99 num_samples 5800 loss 0.00519657593153086
Epoch 99 num_samples 5900 loss 0.00643591098127574
Epoch 99 num_samples 6000 loss 0.005469437239966432
Epoch 99 num_samples 6100 loss 0.005081002908745278
Epoch 99 num_samples 6200 loss 0.005542675869957562
Epoch 99 num_samples 6300 loss 0.006795489431706988
Epoch 99 num_samples 6400 loss 0.004502112580190431
Epoch 99 num_samples 6500 loss 0.003885827047878846
Epoch 99 num_samples 6600 loss 0.009280595107831321
Epoch 99 num_samples 6700 loss 0.004833789424508882
Epoch 99 num_samp

label: 6 predict: 6
label: 9 predict: 9
label: O predict: O
label: 9 predict: 9
label: 1 predict: 1
label: 7 predict: 7
label: 8 predict: 8
label: 3 predict: 3
label: 3 predict: 3
label: 2 predict: 2
label: 5 predict: 5
label: O predict: O
label: 7 predict: 7
label: 6 predict: 6
label: 2 predict: 2
label: 5 predict: 5
label: 4 predict: 4
label: 9 predict: 9
label: 3 predict: 3
label: 5 predict: 5
label: 7 predict: 7
label: 2 predict: 2
label: 7 predict: 7
label: 1 predict: 1
label: 8 predict: 8
label: Z predict: Z
label: 6 predict: 6
label: Z predict: Z
label: 8 predict: 8
label: O predict: O
label: 1 predict: 1
label: 8 predict: 8
label: 3 predict: 3
label: 4 predict: 4
label: 1 predict: 1
label: 7 predict: 7
label: 8 predict: 8
label: 6 predict: 6
label: 1 predict: 1
label: 8 predict: 8
label: 3 predict: 3
label: 9 predict: 9
label: 2 predict: 2
label: O predict: O
label: Z predict: Z
label: 1 predict: 1
label: 3 predict: 3
label: O predict: O
label: 1 predict: 1
label: 5 predict: 5
